### ALL THE IMPORTS

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent, SocietyOfMindAgent
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.messages import  TextMessage
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

### API KEYS

In [3]:
api_key = os.getenv('GOOGLE_API_KEY')

In [4]:
model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": True,
    }

In [5]:
# api_key = os.getenv('OPENAI_API_KEY')
# base_url = os.getenv('BASE_URL')

### MODEL CLIENT

In [6]:
model_client = OpenAIChatCompletionClient(model='gemini-2.5-pro',model_info=model_info,api_key=api_key)

#### This asynchronous function simulates the execution of provided Python code in a safe, isolated environment. It uses exec to run the code, captures any 'result' variable from the locals, or returns a success message if none. If an error occurs, it catches the exception and returns an error string. It's wrapped as a FunctionTool for use in agent workflows, allowing code testing without full runtime risks.

In [ ]:
async def execute_code(code: str) -> str:
    """Simulate code execution and return result."""
    try:
        exec_locals = {}
        exec(code, {}, exec_locals)
        return str(exec_locals.get('result', 'Execution successful.'))
    except Exception as e:
        return f"Error: {str(e)}"

execute_code_tool = FunctionTool(execute_code, description="Execute Python code and return result.")

### CREATING AGENTS

In [8]:
# Define base agents
code_generator = AssistantAgent(
    name="CodeGenerator",
    model_client=model_client,
    description="Generates initial code for tasks.",
    system_message="You generate Python code for given tasks. Keep it simple and functional.",
    tools=[execute_code_tool],
    reflect_on_tool_use=True
)

code_reviewer = AssistantAgent(
    name="CodeReviewer",
    model_client=model_client,
    description="Reviews code for errors and optimizations.",
    system_message="Review code for bugs, efficiency, and improvements. Suggest fixes. End with 'APPROVED' if good, else 'REVISE'.",
)

code_editor = AssistantAgent(
    name="CodeEditor",
    model_client=model_client,
    description="Edits code based on reviews.",
    system_message="Edit code based on reviewer feedback. Output the revised code.",
)

### ROUND ROBIN TEAM

In [ ]:
# Minimal RoundRobin team
minimal_round_robin_team = RoundRobinGroupChat(
    participants=[code_generator, code_reviewer, code_editor],
    termination_condition=TextMentionTermination("APPROVED"),
    max_turns=20,  # Prevent infinite loops
)

#### SOCIETY OF MIND AGENT (instresting thing is that its and agent wrapping the whole team so after wrapping a round robin team its an agent right and further this agent can be used in teams again like round robin or selector chat or swarm or magenticonegroupchat cool right😂?? )

In [10]:
society_of_mind_agent = SocietyOfMindAgent(
    name="SocietyOfMindCoder",
    team=minimal_round_robin_team,
    model_client=model_client,
    description="Uses an inner RoundRobin team for modular code thinking.",
    instruction="Consult your inner team (CodeGenerator, Reviewer, Editor) to produce high-quality code.",
    response_prompt="Summarize the inner team's final code and output it cleanly.",
)

### THE SELECTOR CHAT TEAM

In [11]:
planner_agent = AssistantAgent(
    name="Planner",
    model_client=model_client,
    description="Decomposes tasks into steps and delegates.",
    system_message="""
Break down coding tasks into steps. For each code-related subtask (e.g., generate, optimize, test code), delegate to SocietyOfMindCoder by addressing it directly in your message, like: 'SocietyOfMindCoder: Generate the initial recursive Fibonacci function.'
Do not write code yourself. Only plan and delegate. After all delegations and responses, summarize and end with 'TERMINATE' when done.
""",
)

### SELECTOR PROMPT (one thing i observed in this whole coding thing is if you want to change llm's behaviour from probabilistic to deterministic strict prompting is must)

In [12]:
selector_prompt = """
You are selecting the next agent to speak.
Available agents: {participants} and their roles {roles}
- Planner: For task decomposition and planning.
- SocietyOfMindCoder: For generating, reviewing, and editing code using modular thinking.

Conversation history: {history}

If the previous message delegates a subtask to a specific agent (e.g., addresses 'SocietyOfMindCoder:'), select that agent.
Otherwise, select the most appropriate agent from {participants} based on the current needs.
Only return the agent name.
"""

### you see i have used the society of mind agent in selector group chat team

In [13]:
hybrid_selector_team = SelectorGroupChat(
    participants=[planner_agent, society_of_mind_agent],
    model_client=model_client,
    termination_condition=TextMentionTermination("TERMINATE"),
    max_turns=10,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow repeated speakers if needed for iteration
)

### PERFORMANCE CHECK ON TASK RUN

In [15]:
task = "Write a Python function to compute Fibonacci sequence up to n, optimize it, and test with n=10."

In [16]:
from autogen_agentchat.ui import Console

In [17]:
await Console(hybrid_selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Write a Python function to compute Fibonacci sequence up to n, optimize it, and test with n=10.
---------- TextMessage (Planner) ----------
This task can be broken down into four main steps:
1.  Create a basic, unoptimized function to calculate a Fibonacci number.
2.  Optimize this function to improve its performance.
3.  Create a main function to generate the sequence up to `n` using the optimized function.
4.  Write test code to run the sequence function for `n=10`.

Let's start with the first step.

SocietyOfMindCoder: Generate a simple recursive Python function named `fib_recursive` that calculates the nth Fibonacci number.
---------- ToolCallRequestEvent (CodeGenerator) ----------
[FunctionCall(id='', arguments='{"code":"def fib_recursive(n):\\n  \\"\\"\\"\\n  Calculates the nth Fibonacci number using recursion.\\n  \\"\\"\\"\\n  if n \\u003c= 1:\\n    return n\\n  else:\\n    return fib_recursive(n-1) + fib_recursive(n-2)\\n\\n# Example of

TaskResult(messages=[TextMessage(id='1882620a-8ec8-4e50-9dd0-ac247e19c165', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 17, 10, 6, 50, 299421, tzinfo=datetime.timezone.utc), content='Write a Python function to compute Fibonacci sequence up to n, optimize it, and test with n=10.', type='TextMessage'), TextMessage(id='ecdf0b4f-0763-491b-aee9-d434e405aead', source='Planner', models_usage=RequestUsage(prompt_tokens=110, completion_tokens=116), metadata={}, created_at=datetime.datetime(2025, 7, 17, 10, 7, 6, 219408, tzinfo=datetime.timezone.utc), content="This task can be broken down into four main steps:\n1.  Create a basic, unoptimized function to calculate a Fibonacci number.\n2.  Optimize this function to improve its performance.\n3.  Create a main function to generate the sequence up to `n` using the optimized function.\n4.  Write test code to run the sequence function for `n=10`.\n\nLet's start with the first step.\n\nSocietyOfMindCoder: Genera

In [ ]:
cancellation_token = CancellationToken()
result = await hybrid_selector_team.run(task=TextMessage(content=task, source="user"), cancellation_token=cancellation_token)
for message in result.messages:
    print(f"{message.source}: {message.content}")
